Connected to Env_Cours12 (Python 3.11.3)

In [1]:
pip install flask

SyntaxError: invalid syntax (<ipython-input-1-d3af629f9f07>, line 2)

In [2]:
from flask import jsonify

@app.route('/api/meteo/')
def meteo():
    dictionnaire = {
        'type' : "Prévision de température",
        'valeurs' : [24, 24, 25, 26, 27, 28],
        'unite' : 'degrés Celsius'
    }
    return jsonify(dictionnaire)

NameError: name 'app' is not defined

In [3]:
from flask import jsonify, Flask, render_template

app = Flask(__name__)


@app.route("/")
def hello():
    return "Hello World!"

@app.route('/dashboard/')
def dashboard():
    return render_template(dashboard.html)

@app.route('/api/meteo/')
def meteo():
    dictionnaire = {
        'type': "Prévision de température",
        'valeurs': [24, 24, 25, 26, 27, 28],
        'unite': 'degrés Celsius'
    }
    return jsonify(dictionnaire)

In [4]:
if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
from flask import jsonify, Flask, render_template

app = Flask(__name__)


@app.route("/")
def hello():
    return "Hello World!"

@app.route('/dashboard/')
def dashboard():
    return render_template(dashboard.html)

@app.route('/api/meteo/')
def meteo():
    dictionnaire = {
        'type': "Prévision de température",
        'valeurs': [24, 24, 25, 26, 27, 28],
        'unite': 'degrés Celsius'
    }
    return jsonify(dictionnaire)

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

Connected to Env_Cours12 (Python 3.11.3)

In [1]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            mlflow.log_artifacts('Artifacts')
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

NameError: name 'mlflow' is not defined

In [2]:
import os
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
# import mlflow_tools => en attendant de régler le problème.

In [3]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [4]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [5]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [6]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [7]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            mlflow.log_artifacts('Artifacts')
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


In [8]:
import os
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
# import mlflow_tools => en attendant de régler le problème.

In [9]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [10]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [11]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [12]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [13]:
'''os.makedirs("Artifacts", exist_ok=True)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)
with open("Artifacts/features.txt", "w") as f:
    f.write(features)'''

art_features = X_test.columns
art_data = X_test.to_json()

In [14]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alterna

In [15]:
import os
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
# import mlflow_tools => en attendant de régler le problème.

In [16]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [17]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [18]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [19]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [20]:
art_features = X_test.columns
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

TypeError: write() argument must be str, not Index

In [ ]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

: 

In [21]:
art_features = X_test.columns()
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

TypeError: 'Index' object is not callable

In [22]:
art_features = X_test.columns.values
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

TypeError: write() argument must be str, not numpy.ndarray

In [23]:
art_features = X_test.columns.values
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

<class 'numpy.ndarray'>


TypeError: write() argument must be str, not numpy.ndarray

In [24]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = art_features.tostring()
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

<ipython-input-24-15a1796bb4a8>:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  art_features = art_features.tostring()


TypeError: write() argument must be str, not bytes

In [25]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

NameError: name 'json' is not defined

In [26]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
# import mlflow_tools => en attendant de régler le problème.

In [27]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [28]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [29]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [30]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [31]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [32]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


In [33]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
# import mlflow_tools => en attendant de régler le problème.

In [34]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [35]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [36]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [37]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [38]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [39]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [40]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
            
            # Model signature
            signature = infer_signature(X_test, predictions)
            
            # Model logging
            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = 'Artifact',
                signature=signature,
                
                registered_modoel_name="sk-learn-logistic-reg-model"
            )
            
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


NameError: name 'infer_signature' is not defined

In [41]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
from mlflow import infer_signature
# import mlflow_tools => en attendant de régler le problème.

ImportError: cannot import name 'infer_signature' from 'mlflow' (c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\mlflow\__init__.py)

In [42]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [43]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [44]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [45]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [46]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [47]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
            
            # Model signature
            signature = infer_signature(X_test, predictions)
            
            # Model logging
            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = 'Artifact',
                signature=signature,
                
                registered_modoel_name="sk-learn-logistic-reg-model"
            )
            
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.56
	F1_score: 0.14
	Coût métier: 0.6
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


NameError: name 'infer_signature' is not defined

In [48]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
from mlflow.models import infer_signature
# import mlflow_tools => en attendant de régler le problème.

In [49]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [50]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [51]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [52]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [53]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [54]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [55]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
            
            # Model signature
            signature = infer_signature(X_test, predictions)
            
            # Model logging
            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = 'Artifact',
                signature=signature,
                
                registered_modoel_name="sk-learn-logistic-reg-model"
            )
            
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\mlflow\models\signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None


TypeError: log_model() got an unexpected keyword argument 'registered_modoel_name'

In [56]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
            
            # Model signature
            signature = infer_signature(X_test, predictions)
            
            # Model logging
            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = 'Artifact',
                signature=signature,
                registered_model_name="sk-learn-logistic-reg-model"
            )
            
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\mlflow\models\signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Successfully registered model 'sk-learn-logistic-reg-model'.
2023/10/12 09:56:25 INFO mlflow.tracking._model_registry.client: 

In [57]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
from mlflow.models import infer_signature
# import mlflow_tools => en attendant de régler le problème.

In [58]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)

In [59]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [60]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [61]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [62]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [63]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            # accuracy, auc, f1 = mlflow_tools.mlflow_eval_metrics(y_test, predictions)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # mlflow_tools.mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc, f1)
            mlflow_log(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Tracking / arttifacts logging
            tracking_uri = mlflow.get_tracking_uri()
            print(f"Current tracking uri: {tracking_uri}")
            
            # Artifact logging
            mlflow.log_artifacts('Artifacts')
            
            # Model signature
            signature = infer_signature(X_test, predictions)
            
            # Model logging
            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = 'Artifact',
                signature=signature,
                registered_model_name="sk-learn-logistic-reg-model"
            )
            
    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log
            mlflow_log(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61
Current tracking uri: file:///c:/Users/amcuc/OCR%20projects/Project%207/mlruns


c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\mlflow\models\signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\_distutils_hack\__init__.py

In [64]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
from mlflow.models import infer_signature
# import mlflow_tools => en attendant de régler le problème.

In [65]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log_experiment(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)
    

def mlflow_log_model(model, model_registration, signature, artifact_path="Artifacts")
    # Artifact logging
    mlflow.log_artifacts(artifact_path)

    # Model signature
    signature = infer_signature(X_test, predictions)
            
    # Model logging
    mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = artifact_path,
        signature=signature,
        registered_model_name="sk-learn-logistic-reg-model"
    )
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

SyntaxError: expected ':' (<ipython-input-65-974ba9e52bed>, line 90)

In [66]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [67]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [68]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [69]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [70]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [71]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [72]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            mlflow_log_experiment(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Model logging
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-logistic-reg-model", signature)

    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-random-forest-model", signature)
            
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
            
            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-gradient-boosting-model", signature)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

NameError: name 'mlflow_log_experiment' is not defined

In [73]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
from mlflow.models import infer_signature
# import mlflow_tools => en attendant de régler le problème.

In [74]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log_experiment(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)
    

def mlflow_log_model(model, model_registration, signature, artifact_path="Artifacts")
    # Artifact logging
    mlflow.log_artifacts(artifact_path)

    # Model signature
    signature = infer_signature(X_test, predictions)
            
    # Model logging
    mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = artifact_path,
        signature=signature,
        registered_model_name="sk-learn-logistic-reg-model"
    )
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

SyntaxError: expected ':' (<ipython-input-74-974ba9e52bed>, line 90)

In [75]:
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    '''Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    '''
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log_experiment(model, run_name, params, accuracy, auc, f1, cout_metier):
    '''Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    '''
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)
    

def mlflow_log_model(model, model_registration, signature, artifact_path="Artifacts"):
    # Artifact logging
    mlflow.log_artifacts(artifact_path)

    # Model signature
    signature = infer_signature(X_test, predictions)
            
    # Model logging
    mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = artifact_path,
        signature=signature,
        registered_model_name="sk-learn-logistic-reg-model"
    )
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

In [76]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [77]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [78]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [79]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [80]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            mlflow_log_experiment(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Model logging
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-logistic-reg-model", signature)

    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-random-forest-model", signature)
            
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
            
            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-gradient-boosting-model", signature)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61


c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'sk-learn-logistic-reg-model' already exists. Creating a new version of this model...
2023/10/12 10:44:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to fin

Model saved in run d2da26347ec641439821d3ae93d86bc3


Created version '3' of model 'sk-learn-logistic-reg-model'.


In [81]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            mlflow_log_experiment(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Model logging
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-logistic-reg-model", signature)

    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-random-forest-model", signature)
            
    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)
            
            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-gradient-boosting-model", signature)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61


c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'sk-learn-logistic-reg-model' already exists. Creating a new version of this model...
2023/10/12 10:44:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to fin

Model saved in run b07a4611baa14a84bdd31c18eb950b02


Created version '4' of model 'sk-learn-logistic-reg-model'.


In [82]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
from mlflow.models import infer_signature
import mlflow_tools #=> en attendant de régler le problème.


'''# %% Fonctions copiées/collées ici en attendant de résoudre le problème de dépendance
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log_experiment(model, run_name, params, accuracy, auc, f1, cout_metier):
    Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)
    

def mlflow_log_model(model, model_registration, signature, artifact_path="Artifacts"):
    # Artifact logging
    mlflow.log_artifacts(artifact_path)

    # Model signature
    signature = infer_signature(X_test, predictions)
            
    # Model logging
    mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = artifact_path,
        signature=signature,
        registered_model_name="sk-learn-logistic-reg-model"
    )
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
'''

'# %% Fonctions copiées/collées ici en attendant de résoudre le problème de dépendance\ndef mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):\n    Calculates classification metrics.\n\n    -------------\n    Parameters:\n    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.\n    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.\n    - fn_to_fp : int : Pondération du coût d\'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.\n    \n    -------------\n    Returns:\n    - accuracy : float : Fraction of correctly classified samples.\n    - auc : float : Area Under the Curve.\n    - f1 : float : F1 score of the positive class in binary classification.\n    - cout_metier : \n    \n    accuracy = accuracy_score(y_true, y_pred)\n    f1 = f1_score(y_true, y_pred)\n    fpr, tpr, _ = roc_curve(y_true, y_pred)\n    auc_ = auc(fpr, tpr)\n    # Calcu

In [83]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [84]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [85]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [86]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [87]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [88]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            mlflow_log_experiment(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Model logging
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-logistic-reg-model", signature)

    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-random-forest-model", signature)

    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédiction et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-gradient-boosting-model", signature)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

Logistic Regression avec dataset under-sampled
	Accuracy: 0.74
	AUC: 0.55
	F1_score: 0.13
	Coût métier: 0.61


c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\mlflow\models\signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\_distutils_hack\__init__.py

Model saved in run d6e016684dd546cdad30dc744b1e206b


Created version '5' of model 'sk-learn-logistic-reg-model'.


Restarted Env_Cours12 (Python 3.11.3)

In [1]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix

import mlflow
from mlflow.models import infer_signature
from mlflow_tools import mlflow_eval_metrics, mlflow_log_experiment, mlflow_log_model 


'''# %% Fonctions copiées/collées ici en attendant de résoudre le problème de dépendance
def mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):
    Calculates classification metrics.

    -------------
    Parameters:
    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.
    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.
    - fn_to_fp : int : Pondération du coût d'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.
    
    -------------
    Returns:
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    - cout_metier : 
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_ = auc(fpr, tpr)
    # Calcul d'un coût métier pondérant un Faux négatif à 10 fois un Faux positif.
    # Coût métier = (Nombre de faux négatifs x pondération + Nombre de faux positifs x 1) / nombre de prédictions 
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cout_metier = (fn * fn_to_fp + fp * 1)/y_pred.shape[0]

    return accuracy, auc_, f1, cout_metier


def mlflow_log_experiment(model, run_name, params, accuracy, auc, f1, cout_metier):
    Logs a run via MLFlow.

    -------------
    Parameters:
    - model : sklearn model : Instance of a sklearn model among:
                                - sklearn.ensemble.RandomForestClassifier
                                - skleanr.ensemble.GradientBoostingClassifier
                                - sklearn.linear_model.LogisticRegression
    - run_name : str : Name of the MLFlow run.
    - params: dictionnaire : Dictionary of params as expressed by the cross_validation hyper parameter optimizer .best_params_.
                        - RandomForestClassifier: ('max_depth', 'min_samples_split', 'n_estimators')
                        - LogisticRegression: ('penalyt', 'C')
    - accuracy : float : Fraction of correctly classified samples.
    - auc : float : Area Under the Curve.
    - f1 : float : F1 score of the positive class in binary classification.
    
    if isinstance(model, RandomForestClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('max_depth', params['max_depth'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])
        mlflow.log_param('n_estimators', params['n_estimators'])

        # mlflow.sklearn.save_model(model, "RandomForestClassifier1")

    elif isinstance(model, LogisticRegression):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('C', params['C'])

        # mlflow.sklearn.save_model(model, "LogisticRegression")

    elif isinstance(model, GradientBoostingClassifier):
        print(run_name)
        print(f"\tAccuracy: {round(accuracy,2)}")
        print(f"\tAUC: {round(auc,2)}")
        print(f"\tF1_score: {round(f1,2)}")
        print(f"\tCoût métier: {round(cout_metier,2)}")

        mlflow.log_param('learning_rate', params['learning_rate'])
        mlflow.log_param('n_estimators', params['n_estimators'])
        mlflow.log_param('min_samples_split', params['min_samples_split'])

        # mlflow.sklearn.save_model(model, "GradientBoostingRegressor")
        
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('auc', auc)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('coût_métier', cout_metier)
    

def mlflow_log_model(model, model_registration, signature, artifact_path="Artifacts"):
    # Artifact logging
    mlflow.log_artifacts(artifact_path)

    # Model signature
    signature = infer_signature(X_test, predictions)
            
    # Model logging
    mlflow.sklearn.log_model(
        sk_model = model,
        artifact_path = artifact_path,
        signature=signature,
        registered_model_name="sk-learn-logistic-reg-model"
    )
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
'''

'# %% Fonctions copiées/collées ici en attendant de résoudre le problème de dépendance\ndef mlflow_eval_metrics(y_true, y_pred, fn_to_fp = 10):\n    Calculates classification metrics.\n\n    -------------\n    Parameters:\n    - y_true : 1d array-like, or label indicator array / sparse matrix : Ground truth (correct) labels.\n    - y_pred : 1d array-like, or label indicator array / sparse matrix : Predicted labels, as returned by a classifier.\n    - fn_to_fp : int : Pondération du coût d\'un Faux négatif par rapport à un faux positif. Utilisé pour calculer le coût métier.\n    \n    -------------\n    Returns:\n    - accuracy : float : Fraction of correctly classified samples.\n    - auc : float : Area Under the Curve.\n    - f1 : float : F1 score of the positive class in binary classification.\n    - cout_metier : \n    \n    accuracy = accuracy_score(y_true, y_pred)\n    f1 = f1_score(y_true, y_pred)\n    fpr, tpr, _ = roc_curve(y_true, y_pred)\n    auc_ = auc(fpr, tpr)\n    # Calcu

In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


def addition(a=0, b=0):
    '''Retourne la somme de a et b.
    Fonction uniquement créée pour tester la bonne mise en place de Pytest lors d'un push github.

    Parameters:
        - a : int : Première valeur à additionner.
        - b : int : Deuxième valeur à additionner.

    Returns:
        - somme : int : la somme de a et b.
    '''

    return (a+b)

In [3]:
method = "Logistic Regression"  # à revoir. Semble s'appeler via le terminal avec des paramètres => method: str = sys.argv[1] if len(sys.argv) > 1 else '
dataset = "under-sampled" # "base "ou "under-sampled" ou "over-sampled"
run_name = method + " avec dataset " + dataset
print(run_name)

Logistic Regression avec dataset under-sampled


In [4]:
# Train set
if dataset == "base":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train.parquet'))
elif dataset == "over-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_oversampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_oversampled.parquet'))
elif dataset == "under-sampled":
    X_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_train_undersampled.parquet'))
    y_train = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_train_undersampled.parquet'))

# Test set
X_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'X_test.parquet'))
y_test = pd.read_parquet(os.path.join('Dataset', 'Data clean', 'y_test.parquet'))

In [5]:
art_features = X_test.columns.values
print("art_features:",art_features)
art_features = np.array2string(art_features)
print("art_features:",art_features)
print(type(art_features))
art_data = X_test.to_json()

with open("Artifacts/features.txt", "w") as f:
    f.write(art_features)
with open("Artifacts/data.json", "w", encoding="utf-8") as f:
    json.dump(art_data, f, indent=2)

art_features: ['SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'FLAG_OWN_CAR' 'FLAG_OWN_REALTY'
 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT' 'AMT_ANNUITY'
 'AMT_GOODS_PRICE' 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH'
 'DAYS_EMPLOYED' 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE'
 'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'
 'FLAG_PHONE' 'FLAG_EMAIL' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'HOUR_APPR_PROCESS_START'
 'REG_REGION_NOT_LIVE_REGION' 'REG_REGION_NOT_WORK_REGION'
 'LIVE_REGION_NOT_WORK_REGION' 'REG_CITY_NOT_LIVE_CITY'
 'REG_CITY_NOT_WORK_CITY' 'LIVE_CITY_NOT_WORK_CITY' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'FLOORSMAX_AVG' 'FLOORSMIN_AVG'
 'LANDAREA_AVG' 'LIVINGAPARTMENTS_AVG' 'LIVINGAREA_AVG'
 'NONLIVINGAPARTMENTS_AVG' 'NONLIVINGAREA_AVG' 'APARTMENTS_MODE'
 'BASEMENTAREA_MODE' 'YEARS_BEGI

In [6]:
if __name__ == '__main__':
    mlflow.set_experiment(experiment_name='credit_score_classification')

    if method == "Logistic Regression":
        # Suppression des valeurs manquantes dans le cadre de la régression logistique
        # Valeurs manquantes train
        train = pd.concat([X_train, y_train], axis=1)
        train.dropna(inplace=True)
        y_train = train.pop('TARGET')
        X_train = train
        # Valeurs manquantes test
        test = pd.concat([X_test, y_test], axis=1)
        test.dropna(inplace=True)
        y_test = test.pop('TARGET')
        X_test = test

        # Modélisation Logistic Regression
        # run_name = "Run Logistic regression"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            log_reg = LogisticRegression()

            params_lr = {"C": np.logspace(0.1, 2, 4)}

            halving_grid_lr = HalvingRandomSearchCV(estimator=log_reg, 
                                                    param_distributions=params_lr,
                                                    cv=5,
                                                    scoring='accuracy',
                                                    error_score=0,
                                                    n_jobs=-1
                                                    )
            halving_grid_lr.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = LogisticRegression(**halving_grid_lr.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            mlflow_log_experiment(model, run_name, halving_grid_lr.best_params_, accuracy, auc_, f1, cout_metier)
            
            # Model logging
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-logistic-reg-model", signature)

    elif method == "Random Forest":
        # Modélisation Logistic Regression
        # run_name = "Run Random Forest Classifier"

        with mlflow.start_run(run_name=run_name):
            # HalvingRandomSearchCV
            rfc = RandomForestClassifier()

            params_rfc = {"n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "max_depth": np.linspace(5, 50, 4, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                         }

            halving_rand_rfc = HalvingRandomSearchCV(estimator=rfc, 
                                                     param_distributions=params_rfc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_rfc.fit(X_train, y_train)

            # Modélisation, prédictions et performances
            model = RandomForestClassifier(**halving_rand_rfc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-random-forest-model", signature)

    elif method == "Gradient Boosting":
        # Modélisation Gradient Boosting Classifier
        # run_name = "Run Gradient Boosting Classifier"

        with mlflow.start_run(run_name=run_name):
            # Gradient Boosting
            gbc = GradientBoostingClassifier()

            params_gbc = {"learning_rate": np.logspace(-2, 0, 4),
                          "n_estimators": np.linspace(10, 150, 5, dtype=int),
                          "min_samples_split": np.linspace(5, 50, 4, dtype=int)
                          }

            halving_rand_gbc = HalvingRandomSearchCV(estimator=gbc,
                                                     param_distributions=params_gbc,
                                                     cv=5,
                                                     scoring='accuracy',
                                                     error_score=0,
                                                     n_jobs=-1
                                                    )
            halving_rand_gbc.fit(X_train, y_train)

            # Modélisation, prédiction et performances
            model = GradientBoostingClassifier(**halving_rand_gbc.best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            accuracy, auc_, f1, cout_metier = mlflow_eval_metrics(y_test, predictions)

            # MLFlow log experiment
            mlflow_log_experiment(model, run_name, halving_rand_rfc.best_params_, accuracy, auc_, f1, cout_metier)

            #MLFlow log model
            signature = infer_signature(X_test, predictions)
            mlflow_log_model(model, "sk-learn-gradient-boosting-model", signature)

c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=58. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\amcuc\OCR projects\Cours\12. Déployez une API de prédiction\Env_Cours12\Lib\site-packages\sklearn\linear_model\_logistic.py:460:

NameError: name 'roc_curve' is not defined

No kernel connected